# Explorando o algoritmo A* (U2T2)
---
Disciplina: Algoritmos e Estruturas de Dados II
</br>Discente: Felipe Gabriel B. da Silva - 20240001322

---
</br>
O presente projeto tem como objetivo analisar o algoritmo A* e comparar com os algoritmos de Dijkstra original e Dijkstra + Min-heap. Para o trabalho foi necessário analisar as melhores rotas para que 10 colaboradores conseguissem atender os 65 locais espalhados pela cidade de Natal/RN, saindo de um local central, realizando as coletas e retornando ao local, buscando minimizar a distancia percorrida.

</br>
Bibliotecas importantes que são utilizadas no projeto:

*   Matplotlib
*   Networkx
*   Osmnx
*   Pandas
*   Sklearn.cluster
*   Heapq
*   Time
*   Codecarbon
*   IPython.display
</br>

In [ ]:
!pip install --upgrade osmnx
!pip install codecarbon

In [ ]:
import osmnx as ox
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import networkx as nx
import heapq
import time
from codecarbon import EmissionsTracker
from IPython.display import display

In [ ]:
# Inicializa grafo vazio
G = None  # Cria a variável G sem nenhum grafo atribuído inicialmente

# Baixa o grafo viário da cidade de Natal/RN, considerando apenas vias para veículos
G_bairro = ox.graph_from_place('Natal, Brazil', network_type='drive')

G = G_bairro if G is None else nx.compose(G, G_bairro)


In [ ]:
# impute missing edge speeds and calculate edge travel times with the speed module
G = ox.routing.add_edge_speeds(G)
G = ox.routing.add_edge_travel_times(G)

est_central = ox.nearest_nodes(G, X=-35.2621671,Y=-5.7531857)

arquivo = pd.read_csv('/content/drive/MyDrive/2025.1/AED2/T2U2/centroid_filtered.csv')
# 3. Mapear coordenadas aos nós mais próximos

arquivo['node'] = arquivo.apply(lambda row: ox.distance.nearest_nodes(G, row['Lat'], row['Lon']), axis=1)

coords = arquivo[['Lat','Lon']].values

kmeans = KMeans(n_clusters=10, random_state=42).fit(coords)

arquivo['grupo'] = kmeans.labels_

# Algoritmo Dijkstra

In [ ]:
# Emissão de CO2
tracker_dijkstra = EmissionsTracker(project_name='Dijkstra')
tracker_dijkstra.start()

inicio_dijkstra = time.time()

rotas_dijkstra = []
dist_total = 0
print(dist_total)
for i in range(10):
    grupo = arquivo[arquivo['grupo'] == i]
    nodes = grupo['node'].tolist()

    rota = [est_central]
    atual = est_central
    visitados = set()

    dist = 0
    while len(visitados) < len(nodes):
        menor_dist = float("inf")
        proximo = None
        for destino in nodes:
            if destino not in visitados:
                try:
                    d = nx.shortest_path_length(G, atual, destino, weight='length')
                    if d < menor_dist:
                        menor_dist = d
                        proximo = destino
                except:
                    continue
        if proximo:
            caminho = nx.shortest_path(G, atual, proximo, weight='length')
            rota.extend(caminho[1:])
            dist += menor_dist
            visitados.add(proximo)
            atual = proximo

    # Voltar para a estação
    try:
        caminho_volta = nx.shortest_path(G, atual, est_central, weight='length')
        rota.extend(caminho_volta[1:])
        dist += nx.shortest_path_length(G, atual, est_central, weight='length')
    except:
        print(f"Não foi possível voltar para a estação no grupo {i}")

    rotas_dijkstra.append({'grupo': i, 'rota': rota, 'distancia': dist})
    dist_total += dist
dist_total_dijkstra = dist_total
print(dist_total_dijkstra)
fim_dijkstra = time.time()

duracao_dijkstra = fim_dijkstra - inicio_dijkstra

tracker_dijkstra.stop()

# 7. Plotar as rotas
ox.plot_graph_routes(
    G,
    [r['rota'] for r in rotas_dijkstra],
    route_colors=ox.plot.get_colors(n=10),
    route_linewidth=2,
    node_size=0,
    figsize=(15,15)
)

# Algoritmo A_star

In [ ]:
# impute missing edge speeds and calculate edge travel times with the speed module
G = ox.routing.add_edge_speeds(G)
G = ox.routing.add_edge_travel_times(G)

est_central = ox.nearest_nodes(G, X=-35.2621671,Y=-5.7531857)

arquivo = pd.read_csv('/content/drive/MyDrive/2025.1/AED2/T2U2/centroid_filtered.csv')
# 3. Mapear coordenadas aos nós mais próximos

arquivo['node'] = arquivo.apply(lambda row: ox.distance.nearest_nodes(G, row['Lat'], row['Lon']), axis=1)

coords = arquivo[['Lat','Lon']].values

kmeans = KMeans(n_clusters=10, random_state=42).fit(coords)

arquivo['grupo'] = kmeans.labels_

tracker_astar = EmissionsTracker(project_name='A_star')
tracker_astar.start()

inicio_astar = time.time()

rotas_astar = []
print('\n',dist_total)
dist_total = 0
print('\n',dist_total)
# Heurística com great_circle do OSMnx
heuristica = lambda u, v: ox.distance.great_circle(
    G.nodes[u]['y'], G.nodes[u]['x'],
    G.nodes[v]['y'], G.nodes[v]['x']
)

for i in range(10):
    grupo = arquivo[arquivo['grupo'] == i]
    nodes = grupo['node'].tolist()

    rota = [est_central]
    atual = est_central
    visitados = set()

    dist = 0
    while len(visitados) < len(nodes):
        menor_dist = float("inf")
        proximo = None
        for destino in nodes:
            if destino not in visitados:
                try:
                    d = nx.astar_path_length(G, atual, destino, weight='length', heuristic=heuristica)
                    if d < menor_dist:
                        menor_dist = d
                        proximo = destino
                except:
                    continue
        if proximo:
            caminho = nx.astar_path(G, atual, proximo, weight='length', heuristic=heuristica)
            rota.extend(caminho[1:])
            dist += menor_dist
            visitados.add(proximo)
            atual = proximo

    # Voltar para a estação
    try:
        caminho_volta = nx.astar_path(G, atual, est_central, weight='length', heuristic=heuristica)
        rota.extend(caminho_volta[1:])
        dist += nx.astar_path_length(G, atual, est_central, weight='length', heuristic=heuristica)
    except:
        print(f"Não foi possível voltar para a estação no grupo {i}")

    rotas_astar.append({'grupo': i, 'rota': rota, 'distancia': dist})
    dist_total += dist

dist_total_astar = dist_total
print('\n',dist_total_astar)
fim_astar = time.time()
duracao_astar = fim_astar - inicio_astar

tracker_astar.stop()

# 7. Plotar as rotas
ox.plot_graph_routes(
    G,
    [r['rota'] for r in rotas_astar],
    route_colors=ox.plot.get_colors(n=10),
    route_linewidth=2,
    node_size=0,
    figsize=(15,15)
)

# Dijkstra + min-heap


In [ ]:
# impute missing edge speeds and calculate edge travel times with the speed module
G = ox.routing.add_edge_speeds(G)
G = ox.routing.add_edge_travel_times(G)

est_central = ox.nearest_nodes(G, X=-35.2621671,Y=-5.7531857)

arquivo = pd.read_csv('/content/drive/MyDrive/2025.1/AED2/T2U2/centroid_filtered.csv')
# 3. Mapear coordenadas aos nós mais próximos

arquivo['node'] = arquivo.apply(lambda row: ox.distance.nearest_nodes(G, row['Lat'], row['Lon']), axis=1)

coords = arquivo[['Lat','Lon']].values

kmeans = KMeans(n_clusters=10, random_state=42).fit(coords)

arquivo['grupo'] = kmeans.labels_

tracker_minheap = EmissionsTracker(project_name='Dijkstra Min-Heap')
tracker_minheap.start()

inicio_minheap = time.time()

# 3. Dijkstra com heapq (min-heap)
def dijkstra_heap(G, origem, destino):
    heap = [(0, origem)]
    visitados = set()
    distancias = {origem: 0}

    while heap:
        custo, atual = heapq.heappop(heap)
        if atual == destino:
            return custo
        if atual in visitados:
            continue
        visitados.add(atual)
        for vizinho in G.neighbors(atual):
            peso = G[atual][vizinho][0]['length']
            novo_custo = custo + peso
            if vizinho not in distancias or novo_custo < distancias[vizinho]:
                distancias[vizinho] = novo_custo
                heapq.heappush(heap, (novo_custo, vizinho))
    return float('inf')

# 4. Reconstruir caminho da rota
def reconstruir_caminho(G, origem, destino):
    heap = [(0, origem, [origem])]
    visitados = set()

    while heap:
        custo, atual, caminho = heapq.heappop(heap)
        if atual == destino:
            return caminho
        if atual in visitados:
            continue
        visitados.add(atual)
        for vizinho in G.neighbors(atual):
            peso = G[atual][vizinho][0]['length']
            heapq.heappush(heap, (custo + peso, vizinho, caminho + [vizinho]))
    return []

# 5. Geração das rotas com Dijkstra Heap
rotas_minheap = []
print('\n',dist_total)
dist_total = 0
print('\n',dist_total)
for i in range(10):
    grupo = arquivo[arquivo['grupo'] == i]
    nodes = grupo['node'].tolist()

    rota = [est_central]
    atual = est_central
    visitados = set()
    dist = 0

    while len(visitados) < len(nodes):
        menor_dist = float('inf')
        proximo = None
        for destino in nodes:
            if destino not in visitados:
                try:
                    d = dijkstra_heap(G, atual, destino)
                    if d < menor_dist:
                        menor_dist = d
                        proximo = destino
                except:
                    continue
        if proximo:
            caminho = reconstruir_caminho(G, atual, proximo)
            rota.extend(caminho[1:])
            dist += menor_dist
            visitados.add(proximo)
            atual = proximo

    # Voltar à estação
    try:
        caminho_volta = reconstruir_caminho(G, atual, est_central)
        rota.extend(caminho_volta[1:])
        dist += dijkstra_heap(G, atual, est_central)
    except:
        print(f"Não foi possível voltar para a estação no grupo {i}")

    rotas_minheap.append({'grupo': i, 'rota': rota, 'distancia': dist})
    dist_total += dist

dist_total_minheap = dist_total
print('\n',dist_total_minheap)
fim_minheap = time.time()

duracao_minheap = fim_minheap - inicio_minheap

tracker_minheap.stop()

# 6. Plotar rotas
ox.plot_graph_routes(
    G,
    [r['rota'] for r in rotas_minheap],
    route_colors=ox.plot.get_colors(n=10),
    route_linewidth=2,
    node_size=0,
    figsize=(15, 15)
)

# Comparando os resultados

In [ ]:
# Lê o arquivo gerado pelo CodeCarbon
df_emissions = pd.read_csv('/content/emissions.csv')

# Selecionar os últimos 3 registros (um para cada algoritmo)
ultimas_emissoes = df_emissions.tail(3)['emissions'].tolist()

# Agora você pode usar os valores:5w
co2_dijkstra = ultimas_emissoes[0]
co2_minheap = ultimas_emissoes[1]
co2_astar = ultimas_emissoes[2]

dados = {
    'Algoritmo': ['Dijkstra Padrão', 'Dijkstra Min-Heap', 'A*'],
    'Tempo de Execução (s)': [duracao_dijkstra, duracao_minheap, duracao_astar],
    'Distância Total (km)': [dist_total_dijkstra, dist_total_minheap, dist_total_astar],
    'Emissão CO₂ (g)': [co2_dijkstra, co2_minheap, co2_astar]
}

df_comparativo = pd.DataFrame(dados)

# Formatado para 2 casas decimais
df_formatado = df_comparativo.copy()
df_formatado['Tempo de Execução (s)'] = df_formatado['Tempo de Execução (s)'].map(lambda x: f"{x:.2f}")
df_formatado['Distância Total (km)'] = df_formatado['Distância Total (km)'].map(lambda x: f"{x:.2f}")
df_formatado['Emissão CO₂ (g)'] = df_formatado['Emissão CO₂ (g)'].map(lambda x: f"{x:.6f}")

display(df_comparativo)